# Time Series Anomalies Exercises

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import wrangle as wr
from env import host, username, password

The dataset for these exercises lives in the Codeup Data Science MySQL Server. The database name is curriculum_logs.

Go through the lesson commenting code, adding docstrings, and adding markdown to support what is happening.

```sql
query = '''
        SELECT *
        FROM logs
        '''
```

- This turns our SQL query into a variable to be used in the function to retrieve the data

```python
url = wr.get_connection('curriculum_logs')
```

- This will turn the calling of our function to get the data into a variable name

```python
df = pd.read_sql(query, url)
df.head()
```

- This will run the function to collect the data and turn it into a dataframe for use

```python
df = pd.read_csv('curriculum_logs.csv')
```

- This will turn that just created dataframe into a csv within the directory so that we don't need to run the query again.

```python
df['date'] = pd.to_datetime(df['date'], infer_datetime_format=True)
df.dtypes
```

- This will turn the date column into a datetime data type so that we can convert it into the index.

```python
df = df.set_index('date')
```

- This will actually set the date column to the index

```python
daily_hits = df['path'].resample('d').count()
```

This will create the variable daily_hits to be the dataframe, resampled by the 'path' column, and turned into a value count of for each day

```python
weekly_avg = daily_hits.ewm(span=7).mean()
```

This will create the exponential moving average (ewm) from the daily_hits series. You must define the span to use here, which, since we resampled by each day, making the span 7 will create the exponential moving average to be calculated by the last 7 days. It will then turn that into an average

```python
monthly_avg = daily_hits.ewm(span=30).mean()
```

The same as above, but for a month instead of a week

```python
quarterly_avg = daily_hits.ewm(span=90).mean()
```

Also the same as above, but for a quarter (90 days)

```python
plt.figure(figsize=(13, 7))
plt.plot(daily_hits, label='Original')
plt.plot(weekly_avg, label='Weekly')
plt.plot(monthly_avg, label='Monthly')
plt.plot(quarterly_avg, label='Quarterly')
plt.xlabel('Date')
plt.ylabel('Number of hits')
plt.title('EMA of website hits over time')
plt.legend()
plt.show()
```

This will basically plot each of those three averages against the original data, so that we can visualize how each is compared to the others

Now onto Bollinger bands, which are used to indicate volatility in price over time. The three parts of the bollinger bands are the lower band, the upper band, and the midband.

The upper and lower bands are calculated by:

midband +/- standard deviation * k

`+` for the upper band

`-` for the lower band

```python
monthly_std = daily_hits.ewm(span=30).std()
```

Doing very similar as above, but instead of getting the mean for the emw we are getting the standard deviation

```python
upper_band = monthly_avg + monthly_std * 1.5
lower_band = monthly_avg - monthly_std * 1.5
```

Getting the upper and lower bands; equation is very similar to what we use to determine the fences for the `iqr`

```python
final_df = pd.concat([daily_hits, monthly_avg, upper_band, lower_band], axis=1)
```

Creating and concatenating onto a final dataframe with each piece of information to use and plot the bands

```python
final_df.columns = ['hits', 'midband', 'upper_band', 'lower_band']
```

Renaming the columns in the newly created dataframe so that they are accurate and make sense

```python
plt.figure(figsize=(13, 7))
plt.plot(final_df['hits'], label='Original')
plt.plot(final_df['midband'], label='Midband')
plt.plot(final_df['upper_band'], label='Upper Band')
plt.plot(final_df['lower_band'], label='Lower Band')
plt.xlabel('Date')
plt.ylabel('Number of hits')
plt.title('Bollinger bands for curriculum hits')
plt.legend()
plt.show()
```

This will essentially plot out the data with the bolinger bands so you can visually see what they are for each data point.

Now for the %b, which defines where a value lays relative to the created bands.

the formula is:

(last - lower band) / (upper band - lower band)

A value greater than 1 lies above the upper band, and below 0 is below the lower band

```python 
(final_df['hits'] - final_df['lower_band']) / (final_df['upper_band'] - final_df['lower_band'])
```

This is the formula for the %b for our dataset

```python
final_df['pct_b'] = (final_df['hits'] - final_df['lower_band']) / (final_df['upper_band'] - final_df['lower_band'])
```

this is the code to actually use that formula to create a column within the current dataframe that has the actual value for each row

```python
final_df[final_df['pct_b'] > 1].sort_values('pct_b', ascending=False).head()
```

We can use this to filter out all of the values that fall underneath the upper bound; so essentially this will return all of the values that are above the upper bound

```python
final_df[final_df['pct_b'] < 0].sort_values('pct_b').head()
```

Same as above, but for everything below the lower bound

```python
def compute_bollinger(series, col, span=30, k=1.5):
    '''
    Function will take in a resampled series of data, as well as a column name, a span for the ewm method, 
    and k for the fences of our iqr to return a plot for the bollinger bands of the data, 
    as well as a df that holds those values.
    '''
    
    mean_df = series.ewm(span=span).mean()
    std_df = series.ewm(span=span).std()
    
    upper_band = mean_df + std_df * k
    lower_band = mean_df - std_df * k
    
    final_df = pd.concat([series, mean_df, upper_band, lower_band], axis=1)
    
    final_df.columns = [col, 'midband', 'upper_band', 'lower_band']
    
    final_df['pct_b'] = (final_df[col] - final_df['lower_band']) / (final_df['upper_band'] - final_df['lower_band'])
    
    plt.figure(figsize=(13, 7))
    plt.plot(final_df['hits'], label='Original')
    plt.plot(final_df['midband'], label='Midband')
    plt.plot(final_df['upper_band'], label='Upper Band')
    plt.plot(final_df['lower_band'], label='Lower Band')
    
    plt.title(f'Bollinger Bands for Data')
    
    plt.legend()
    plt.show()

    return final_df
```

Function created in class that I added onto to become more useful to basically do everything shown above, aside from resampling the data into a series